In [2]:
from coding import *
import numpy as np

# n=45, s=5, d=1

In [42]:
from scipy import linalg as LA

def obtain_E(alpha, E_2, n, s):
    # obtain E_1 in shape of n-2s by d
    _tmp_y = np.zeros((E_2.shape[1], n-s), dtype=complex)
    _tmp_y[:,0:s] = np.transpose(E_2)[:, -s:]
    for i in range(n-2*s):
        tmp = np.dot(_tmp_y[:,i:s+i], alpha)
        _tmp_y[:,s+i] = tmp.reshape(-1)
    return np.transpose(_tmp_y[:, s:])

def _array_getH(ndarray):
    # get conjugate transpose of a np.ndarray
    return ndarray.conj().T

d=1
n=7
s=2

_poly_a = np.zeros(s+1, dtype=complex)

W, fake_W, W_perp, S, C_1 = search_w(n=n, s=s)
simulated_G = np.random.randn(n, d)
simulated_adv = np.zeros((n, d),dtype=complex)

simulated_adv[0:2,:] = -100+0j

simulated_R = np.dot(W, simulated_G) + simulated_adv
random_vec = np.random.randn(d, 1)

R_1 = np.dot(simulated_R, random_vec)

E_2 = np.dot(W_perp, R_1)

_X_v3 = np.take(E_2, np.array([range(-i-(s+1), -i-2+1) for i in range(s)]))
tmp_y = np.take(E_2, np.array([-i-1 for i in range(s)]), axis=0)

alpha = LA.solve_toeplitz((_X_v3[:,0],_X_v3[0,:]), tmp_y)

_poly_a[0:s] = -alpha.reshape(-1)

recovered_E1 = obtain_E(-alpha, E_2, n, s)
recovered_E = np.concatenate((recovered_E1, E_2), axis=0)

# let's generate true alpha
roots = [np.exp(2*np.pi*i*1j/n) for i in range(s)]
np_poly=P.polyfromroots(roots)

print("recovered E:")
print(recovered_E)
print
print(E_2)
print
print("correct E: ")
print(np.dot(_array_getH(W), simulated_adv))
print("epsilon: ")
print(simulated_adv)

recovered E:
[[ -52.22679793 -2.13162821e-14j]
 [ 127.18451074 +6.12488323e+01j]
 [-191.15936670 -2.39706264e+02j]
 [   2.58603946 +1.13301792e+01j]
 [   2.58603946 -1.13301792e+01j]
 [  20.30262104 -2.54586815e+01j]
 [  42.39483691 -2.04162774e+01j]]

[[  2.58603946+11.33017918j]
 [  2.58603946-11.33017918j]
 [ 20.30262104-25.45868153j]
 [ 42.39483691-20.41627743j]]

correct E: 
[[  1.14121737e+01 -5.00000000e+01j]
 [ -6.04376959e+01 -1.40096887e+02j]
 [ -2.15549609e+02 -9.00968868e+01j]
 [ -2.15549609e+02 +9.00968868e+01j]
 [ -6.04376959e+01 +1.40096887e+02j]
 [  1.14121737e+01 +5.00000000e+01j]
 [  3.20375771e-14 +1.59026902e-14j]]
epsilon: 
[[-100.+0.j]
 [-100.+0.j]
 [   0.+0.j]
 [   0.+0.j]
 [   0.+0.j]
 [   0.+0.j]
 [   0.+0.j]]
